In [1]:
from google.auth import default
credentials, project = default()

In [2]:
project

'swedish-course'

In [6]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import load_json, save_text_file, load_text_file
from src.nlp import get_vocab_dictionary_from_phrases
from src.utils import get_longman_verb_vocab_dict, save_text_file, load_text_file
from src.phrase import generate_phrases_from_vocab_dict, generate_scenario_phrases, generate_scenario_vocab_building_phrases
from src.config_loader import config
from src.gcs_storage import get_phrase_index_path, read_from_gcs, get_phrase_path
COLLECTION = "LM2000"
config.TARGET_LANGUAGE_NAME

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'Swedish'

# Flash Card Generation 01

## Generate english phrases

The core way we store vocabularly for generating phrases, and then flashcards, is in a dictionary with two keys. 'verbs' (for verbs in the infinitive form, like 'be', 'run') and 'vocab' (for everything else).

The intent is that a vocab list is a core learning requirement (e.g. for an exam), and that it is easier to remember words in the context of common phrases. i.e. learning the phrase 'I want', and separtely learning the noun 'cake' is less efficient than learning the phrase 'I want some cake, please'.

Even better if we link that phrase to an image and associated audio. This is the dual-encoding theory of langauge learning and leads to retention and recall benefits.

The first step is generating your english phrases from your vocab list

### Creating conversational phrases from a vocabulary dictionary

This function will iterate through (by sampling) the vocabularly dictionary, until it is exhausted.
We run a check against generated phrases so we can 'tick off' words already used.

Phrases are generated using an LLM

In [ ]:
# or for GCSE vocab:
vocab_dict = load_json("..\data\gcse_vocab_list_cambridge.json")

In [ ]:

#comment out the below two lines to go for the default of 6 - 9 word phrases and no more than 2 verbs
length_phrase = "4-5 words long, for beginner GCSE level, but treat common lexical chunks (I'm going to.., Do you.., Let us.. etc) as a single 'word'"
verbs_per_phrase = "one verb (but OK for an additional auxillary verb if necessary)"
localise = False # whether to tweak the prompt to set phrases within the target country
generated_phrases = generate_phrases_from_vocab_dict(   
    first200_vocab_dict, max_iterations=12,
      length_phrase=length_phrase,
        verbs_per_phrase=verbs_per_phrase,
        localise=localise)
#It takes about 15 iterations to go through 200 verbs, 800 vocab (1000 words total)
#You will end up with about 1000 phrases, so get practice of the same verb etc in different contexts

In [ ]:
generated_phrases = load_text_file('../data/phrases/longman_2000_phrases.txt')

In [ ]:

upload_to_gcs(obj=generated_phrases, bucket_name = config.GCS_PRIVATE_BUCKET, file_name = get_phrase_path(collection = COLLECTION))

In [10]:
get_phrase_path(collection="LM2000")

'collections/LM2000/phrases.json'

In [13]:
generated_phrases = read_from_gcs(config.GCS_PRIVATE_BUCKET,
                                  file_path=get_phrase_path(collection="LM2000"))

RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

In [ ]:
len(set(generated_phrases))

In [ ]:
from src.nlp import create_flashcard_index

WarmUp150_Index = create_flashcard_index(generated_phrases)

In [ ]:
upload_to_gcs(obj=WarmUp150_Index, bucket_name=config.GCS_PRIVATE_BUCKET, file_name = get_phrase_index_path(collection=COLLECTION))

## Remove redundant phrases

In [ ]:
from src.phrase import remove_phrases_with_no_new_words
old_list = load_text_file(phrase_dir / "longman_1000_phrases.txt")
new_list = load_text_file(phrase_dir / "longman_2000_phrases.txt")
new_list_2 = remove_phrases_with_no_new_words(known_phrases=old_list, new_phrases=new_list)

In [ ]:
save_text_file(new_list_2, phrase_dir / "longman_2000_phrases.txt")

In [ ]:
save_text_file(line=ordered_phrase_list, file_path = phrase_dir / "gcse_phrases.txt")

## Generate phrases and vocab for a scenario
Use an LLM to come up with typical phrases for a scenario

In [ ]:
scenario = "meeting new swedish people - language learning community - talking about sweden - hiking, wild swimming, nature"


In [ ]:

speaking_phrases = generate_scenario_phrases(scenario, num_phrases="20 - 25")


In [ ]:

save_text_file(lines=speaking_phrases, file_path=phrase_dir / "swedish_language_learning.txt")

Bulk out this scenarios with some vocab

In [ ]:
vocab_phrases = generate_scenario_vocab_building_phrases(scenario=scenario)
save_text_file(lines=vocab_phrases, file_path=phrase_dir / "swedish_lanuage_learning_vocab2.txt")